In [39]:
import os
import sys
from pathlib import Path
from yaml import load
import shutil as sh
from collections import namedtuple

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
sys.path.insert(0, '../lib')
from ncbi_remap.notebook import Nb
from ncbi_remap.plotting import make_figs

In [4]:
# Setup notebook
nbconfig = Nb.setup_notebook()

# Connect to data store
store = pd.HDFStore('../output/sra.h5', mode='r')

Please check output/fbgn2chrom.tsv. If it does not exist, run bin/fbgn2chrom.py
last updated: 2018-06-18 
Git hash: cbfc1259618cdb40eb1267c1692922d08b50a37a


In [66]:
with open('../aln-wf/patterns.yaml', 'r') as fh:
    patterns = load(fh)

samples = store['aln/complete'].srx.values

In [79]:
uploadSize = 0
failed = []
for srx in samples:
    data_files = [
        Path('../aln-wf', patterns['srxMerge']['bigWig'].format(srx=srx, strand='first')),
        Path('../aln-wf', patterns['srxMerge']['bigWig'].format(srx=srx, strand='first')),
        Path('../aln-wf', patterns['srxMerge']['feature_counts']['counts'].format(srx=srx)),
        Path('../aln-wf', patterns['srxMerge']['feature_counts']['jcounts'].format(srx=srx)),
        Path('../aln-wf', patterns['srxMerge']['feature_counts_intergenic']['counts'].format(srx=srx)),
        Path('../aln-wf', patterns['srxMerge']['feature_counts_intergenic']['jcounts'].format(srx=srx)),
    ]

    try:
        assert np.asarray([x.exists() for x in data_files]).all()
    except AssertionError as e:
        failed.append(srx)
        continue
    
    for f in data_files:
        uploadSize += f.stat().st_size

In [80]:
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti']:
        if abs(num) < 1024.0:
            return f"{num:3.1f}{unit}{suffix}"
        num /= 1024.0

In [81]:
sizeof_fmt(uploadSize)

'1.7TiB'

In [82]:
failed

[]